파이썬의 requests 모듈로 .get으로 서버 호출 가능  
text 메서드로 html 내용 조회가능  
(200은 이상없음, 404 이상있다는 뜻)   
beautifulSoup을 쓰는 이유는 html 문서 파싱 (정리하기, 대량의 데이터 신속하게 수집)  
하지만 대부분 동적 크롤링이니까 selenium  

In [8]:
!pip install selenium
!pip install webdriver_manager

css 선택자와 웹사이트 구조에 대한 경험치
스크롤 다운  


import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

# Configure Chrome options for headless execution
options = Options()
options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-gpu')
options.add_argument('--incognito')

# Set up ChromeDriver via webdriver_manager (works with Selenium 4.x)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Example navigation
driver.get('https://www.naver.com')

# Search for "hello world"
search_input = driver.find_element(By.CLASS_NAME, 'search_input')
search_input.click()
search_input.send_keys('hello world')
search_input.send_keys(Keys.ENTER)

# Wait briefly for results
time.sleep(3)
print(driver.title)

driver.quit()


In [9]:
from IPython.display import display
from selenium import webdriver, __version__ as selenium_version
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

try:
    from selenium.webdriver.chrome.service import Service
except ImportError:
    Service = None

def _supports_service_arg(version_str):
    parts = version_str.split(".")
    try:
        major = int(parts[0])
    except ValueError:
        return True
    if major < 4:
        return False
    if major > 4:
        return True
    minor = 0
    if len(parts) > 1:
        try:
            minor = int(parts[1])
        except ValueError:
            minor = 0
    return minor >= 6

url = "https://securities.koreainvestment.com/main/mall/openels/EdlsInfo.jsp?cmd=TF02cc000000_Main"

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = None
aggregated = []
visited_pages = set()

try:
    if Service is not None and _supports_service_arg(selenium_version):
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
    else:
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    wait = WebDriverWait(driver, 20)
    driver.get(url)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#ifrm_list")))
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#list_style tbody tr")))

    def extract_listing_table():
        table_html = driver.find_element(By.CSS_SELECTOR, "#list_style table").get_attribute("outerHTML")
        try:
            dataframes = pd.read_html(table_html)
        except ValueError:
            return None
        for df in dataframes:
            cleaned = df.dropna(how="all").dropna(axis=1, how="all")
            if cleaned.empty or cleaned.shape[1] < 3:
                continue
            cleaned.columns = [str(col).strip() for col in cleaned.columns]
            return cleaned
        return None

    while True:
        current_page = driver.find_element(By.CSS_SELECTOR, ".pager span.num a.on").text.strip()
        if current_page not in visited_pages:
            table = extract_listing_table()
            if table is not None:
                aggregated.append(table)
            visited_pages.add(current_page)

        page_links = driver.find_elements(By.CSS_SELECTOR, ".pager span.num a")
        next_link = None
        for link in page_links:
            label = link.text.strip()
            if not label.isdigit():
                continue
            if label not in visited_pages:
                next_link = (link, label)
                break

        if next_link is not None:
            marker = driver.find_element(By.CSS_SELECTOR, "#list_style tbody tr:first-child").text
            driver.execute_script("arguments[0].click();", next_link[0])
            wait.until(lambda d: d.find_element(By.CSS_SELECTOR, ".pager span.num a.on").text.strip() == next_link[1])
            wait.until(lambda d: d.find_element(By.CSS_SELECTOR, "#list_style tbody tr:first-child").text != marker)
            continue

        next_button = driver.find_element(By.CSS_SELECTOR, ".pager a.btn.next")
        button_classes = (next_button.get_attribute("class") or "").split()
        if "disabled" in button_classes:
            break

        marker = driver.find_element(By.CSS_SELECTOR, "#list_style tbody tr:first-child").text
        driver.execute_script("arguments[0].click();", next_button)
        wait.until(lambda d: d.find_element(By.CSS_SELECTOR, ".pager span.num a.on").text.strip() not in visited_pages)
        wait.until(lambda d: d.find_element(By.CSS_SELECTOR, "#list_style tbody tr:first-child").text != marker)
        continue
finally:
    if driver is not None:
        try:
            driver.switch_to.default_content()
        except Exception:
            pass
        driver.quit()

if not aggregated:
    raise RuntimeError("No listing data found.")

els_df = pd.concat(aggregated, ignore_index=True)
display(els_df)


C:\Users\Dongjun\AppData\Local\Temp\ipykernel_14160\313178010.py:60: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = pd.read_html(table_html)


TimeoutException: Message: 
